This notebook reads in a geopackage where for each ICESat-2 transect, the id of its overlapping ALS point cloud and the dominant tree species in the transect is saved. It will then clip the ALS point clouds using the transects.

In [1]:
import os
import h5py
import sys


import pandas as pd
import geopandas as gpd
import math
import pdal
from osgeo import ogr
from shapely.geometry import MultiLineString, LineString, Point, Polygon
from shapely import wkt

# for plotting
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import fiona

# for ALS
import subprocess
import json
from shapely.ops import transform
from shapely.validation import explain_validity
from shapely.validation import make_valid
from shapely.ops import transform
import scipy.stats as ss

import geojson
import ee
import geemap as gm

from scipy.stats import median_abs_deviation
import seaborn as sns

In QGIS, using the layer of map squares with forested ICESat-2 polygons in them, and layer with ICESat-2 polygons, a join can be made so that for each polygon the map square it is intersecting will be added. Note that some icesat polygons will be duplicated to different rows when they intersect with more than one map square.

In [3]:
# import icesat polygons with corresponding als map square
geopackage = '..\\Data\\\ICESat2_polygons\\atl08_polygons.gpkg'
layer = 'ATL08_with_ALS_duplicates' # older version was'ATL09_with_ALS'
dataframe = gpd.read_file(geopackage, layer= layer)  

In [5]:
len(dataframe)

298574

In [7]:
# read the .csv that contains the ALS list - this is needed to know the year in the name of the als point cloud
als_list = pd.read_csv('..\\Data\\ALS_files_list.csv')
als_list = als_list.dropna(subset=['ALS_AASTAD'])
als_list

,NR,NR10000,ALS_AASTAD,ALS_TAVA_1,ALS_TAVA_2,ALS_TAVA_3,ALS_TAVA_4,ALS_MADAL,ALS_METS
46,376644,44743,2019,NaN,NaN,2019.0,NaN,NaN,NaN
47,376645,44744,2019,NaN,NaN,2019.0,NaN,NaN,NaN
48,376646,44744,2019,NaN,NaN,2019.0,NaN,NaN,NaN
49,376647,44744,2019,NaN,NaN,2019.0,NaN,NaN,NaN
50,376648,44744,2019,NaN,NaN,2019.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
58195,633631,74331,2013,NaN,2013.0,NaN,NaN,NaN,NaN
58196,633632,74331,"2022,2018,2013,2008",2008.0,2013.0,2018.0,2022.0,NaN,NaN
58197,633633,74331,2013,NaN,2013.0,NaN,NaN,NaN,NaN
58198,633634,74331,2013,NaN,2013.0,NaN,NaN,NaN,NaN


## Clip the ALS laz files

Aout the lastools canopy cover:


  The tool can also produce the canopy cover using option '-cov'.
  The canopy cover is computed as the number of first returns
  above the cover cutoff divided by the number of all first
  returns and output as a percentage. Similarly, with the option
  '-dns' the canopy density can be produced. The canopy density
  is computed as the number of all points above the cover cutoff
  divided by the number of all returns. By default this cover 
  cutoff is identical to the height cutoff. However, using the
  option '-cover_cutoff 5.0' you can set it to a lower or a
  higher value. As the output default percentages between 0.0%
  and 100.0% are produced. Use option '-fractions' to produce
  fractions between 0.000 and 1.000 instead. 

  It is possible to compute the inverse of the canopy cover and
  canopy density with the '-gap' option which will give you 100%
  (or for fractions 1.0) minus canopy cover or canopy density. This is when -gap is included.
  
  -abv is the nr of points above the cutoff heigth. -all is all points.



In [18]:
laszip = r'..\\Data\\tests\\laszip64.exe'
lasclip = r'..\\Data\\tests\\lasclip64.exe' # earlier in the script
lasheight = r'..\\Data\\tests\\lasheight64.exe'
lascan = r'..\\Data\\tests\\lascanopy64.exe'
tools = '..\\P2\\ALSdata\\LAStools_190321\\LAStools\\bin'
lasnoise = r'..\\P2\\ALSdata\\LAStools_190321\\LAStools\\bin\\lasnoise64.exe'
lasinfo = r'..\\Data\\tests\\lasinfo.exe'
las2las = tools + '\\las2las.exe'

In [19]:
# add to the dataframe
dataframe['ALS_av_h']=0.0
dataframe['ALS_98_h'] = 0.0

dataframe['ALS_dns_gap']=0.0
# space to save the ALS lidar metrics
dataframe['SCI'] = 0.0

            


The function does:
1. iterates over the input dataframe - the one with duplicates
2. checks if there are several rows with the same segment number and timestamp, if yes -> UNFINISHED -> should clip the transect from two separate taz files and combine to one. At the moment shows those transects with 0 cap fraction
3. Finds the laz files both for TAVA and METS type 
4. Reads the polygon geometry of the ICESat-2 transect from the dataframe and uses it in lasclip tool to clip the TAVA laz files ---> UNFINISHED -> should do everything for Mets files as well. Save it in a differnt dataframe maby? 
5. Remove all returns above the scan angle threshold and all the noise (removes classes 18, 7, 17 (bridges), 6 (buildings), overlap bit
6. Replace height values with relative height from ground values
7. Calculate from ALS using lastools, the canopy gap, the dns_gap and the average canopy height.
8. Calculate the number of first returns
9. Calculate the 3 canopy metrics FCI, LCI and SCI

# Function

In [45]:
beg = 100000
end = 150000
laz_loc = 'Z:\\Thesis\\Data\\ALS_data\\'

for iindex, row in dataframe[beg:end].iterrows():
    if row['month'] in ['05', '06', '07', '08', '09']:
        typ = 'TAVA'
    else:
        typ = 'METS'
    open('..\\Data\\tests\\test_pol.txt', 'w').close()
    open('..\\Data\\tests\\cover.csv', 'w').close()
    
    laz_file_tava = ''
    laz_file_mets = ''
    laz_file = ''
    
    #change to 0 to avoid bugs
    nr_all_single_ret = 0
    nr_gr_single_ret = 0
    nr_gr_first_ret = 0
    nr_all_first_ret = 0
    nr_gr_last_ret = 0
    nr_all_last_ret = 0
    
    map_sq = int(row['NR'])
    # if the ALS list does not contain this square
    if als_list[als_list['NR'].eq(map_sq)].empty:
        print('Missing point cloud: ', map_sq, '\n')
        continue
    
    segment = row['seg']
    # check if more than one row with this segment nr - first take out the entries that have the same segment nr
    seg = dataframe[dataframe['seg'].eq(segment)]
    
    # thrsholds:
    scan_angl = '10'
    height_cutoff = '2.0'
    
    # find the correct year for TAVA and METS
    tava_year = 0
    mets_year = 0
   
    if pd.isna(als_list[als_list['NR'].eq(map_sq)].iloc[0]['ALS_TAVA_4']):
        # if TAVA_4 is not available, year = to what is in column TAVA_3
        tava_year = int(als_list[als_list['NR'].eq(map_sq)].iloc[0]['ALS_TAVA_3'])
    # else if ALS_TAVA_3 is also na - then give a random year - later this file will not be found and the program continues
    elif pd.isna(als_list[als_list['NR'].eq(map_sq)].iloc[0]['ALS_TAVA_3']):
        tava_year = 9999
    else:
        tava_year = int(als_list[als_list['NR'].eq(map_sq)].iloc[0]['ALS_TAVA_4'])
    if pd.isna(als_list[als_list['NR'].eq(map_sq)].iloc[0]['ALS_METS']):
        mets_year = 9999   
    else:
        mets_year = int(als_list[als_list['NR'].eq(map_sq)].iloc[0]['ALS_METS'])
    
    # find the laz file
   
    if seg[seg['timestamp'].eq(seg.iloc[0]['timestamp'])].shape[0] > 1:    
        # find all rows wiht the same seg ID - if one polygon intersected with two map squares, it is stored on two rows
        # Need to clip each and create a new merged point cloud from the two (?)
        # Implemented elsewhere
        continue

    else:
        laz_file_tava = laz_loc + "TAVA\\" + str(map_sq) + "_" + str(tava_year) + '_tava.laz'
        laz_file_mets = laz_loc + "METS\\" + str(map_sq) + "_" + str(mets_year) + '_mets.laz'
        print(laz_file_tava, '\n', laz_file_mets, ' ', typ)

        if typ = 'METS':
            laz_file = laz_file_mets
        if typ = 'TAVA'
            laz_file = laz_file_tava
        
        # first check if this file exists  
        if os.path.isfile(laz_file):
            pol = dataframe.iloc[iindex]['geometry'][0]
            
            # 1. clip the laz file
            pol_iter = 0
            with open('..\\Data\\tests\\test_pol.txt', 'w') as f:
                while pol_iter < 5:
                    #f.write('Create a new text file!')
                    x = str(pol.exterior.coords[pol_iter][0])
                    y = str(pol.exterior.coords[pol_iter][1])
                    line = x + " " + y + "\n"
                    f.write(line)
                    pol_iter += 1

            polclip = [lasclip,
                        '-i',  laz_file,
                        '-poly', '..\\Data\\tests\\test_pol.txt',
                        '-o',  laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_tava.laz"]            
            pcs =subprocess.Popen(polclip)
            pcs.wait()  
            
            

            # 2. turn laz to las
            testzip= [laszip,
                        '-i', laz_loc + typ + "_transects\\"+ str(segment) + "_" + str(iindex) + "_tava.laz",
                        '-o', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_tava.las"]        
            pcs =subprocess.Popen(testzip)
            pcs.wait()
            
            # 3 remove all returns above the scan angle x and all the noise
            scan_ang = [las2las,
                        '-i',  laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_tava.las",
                        '-drop_class', '6 7 17 18 overlap bit',
                        '-drop_scan_angle_above', scan_angl,
                        '-o', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_tava.las"]       
            pcs =subprocess.Popen(scan_ang)              # start this process
            pcs.wait()

            
            # 4. Replace height values with relative height from ground values
            testheight = [lasheight,
              '-i', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_tava.las",
              '-replace_z',
               '-o', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_h_tava.las"]
            pcs =subprocess.Popen(testheight)
            pcs.wait() 
            
            # 5. Find canopy gap
            testcan= [lascan,
                        '-i', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_h_tava.las",
                        '-height_cutoff', height_cutoff, 
                        '-files_are_plots', 
                        '-p', '98',
                        '-cov',
                        '-dns', 
                        '-gap', 
                        '-avg',
                        '-abv',
                        '-all',
                        '-o', '..\\Data\\tests\\cover.csv']            
            pcs =subprocess.Popen(testcan)
            pcs.wait()
            
            
            # add the answers to the dataframe - but to the icesat all dataframe maybe, not to mapsquares. Or in mapsquares but
            # read canopy cover to temporary dataframe
            try:
                output = pd.read_csv('..\\Data\\tests\\cover.csv')
            except pd.errors.EmptyDataError:
                print('Could not process las file!\n')
                continue
            
            
            dataframe.loc[iindex, 'ALS_dns_gap'] = output['dns_gap'][0]
            dataframe.loc[iindex, 'ALS_av_h'] = output['avg'][0]
            dataframe.loc[iindex, 'ALS_98_h'] = output['p98'][0]
            
            """
            Find variables for gap fraction prediction inidices FCI, LCI, SCI.
            * nr of first returns
            * nr of first ground returns
            
            * nr of last returns
            * nr of last ground
            
            * nr of single returns
            * nr of single ground
            """
            
            # all first, last, single returns
            all_txt = [lasinfo,
                       '-i', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_h_tava.las",
                       '-o', '..\\Data\\tests\\temp_metrics.txt']
            pcs =subprocess.Popen(all_txt)              # start this process
            pcs.wait()
            
            with open('..\\Data\\tests\\temp_metrics.txt') as fh:
                for line in fh:
                    if line.startswith('number of first returns:'):       
                        nr_all_first_ret = [int(s) for s in line.split() if s.isdigit()][0]
                    if line.startswith('number of last returns:'):       
                        nr_all_last_ret = [int(s) for s in line.split() if s.isdigit()][0]
                    if line.startswith('number of single returns:'):       
                        nr_all_single_ret = [int(s) for s in line.split() if s.isdigit()][0]
                        
            # empty file
            open('..\\Data\\tests\\temp_metrics.txt', 'w').close()
            
            # create file with only ground
            only_ground = [las2las,
                        '-i', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + "_h_tava.las",
                        '-keep_class', '2', 
                        '-o', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + '_only_ground.las']        
            pcs =subprocess.Popen(only_ground) 
            pcs.wait() 
            
            gr_txt = [lasinfo,
                       '-i', laz_loc + typ + "_transects\\" + str(segment) + "_" + str(iindex) + '_only_ground.las',
                       '-o', '..\\Data\\tests\\temp_metrics.txt']
            pcs =subprocess.Popen(gr_txt)              # start this process
            pcs.wait()
            
            with open('..\\Data\\tests\\temp_metrics.txt') as fh:
                for line in fh:
                    if line.startswith('number of first returns:'):       
                        nr_gr_first_ret = [int(s) for s in line.split() if s.isdigit()][0]
                    if line.startswith('number of last returns:'):       
                        nr_gr_last_ret = [int(s) for s in line.split() if s.isdigit()][0]
                    if line.startswith('number of single returns:'):       
                        nr_gr_single_ret = [int(s) for s in line.split() if s.isdigit()][0]
                    if line.startswith('number of intermediate returns:'):
                        nr_gr_interm_ret = [int(s) for s in line.split() if s.isdigit()][0]




            
            if nr_all_single_ret != 0:
                
                dataframe.loc[iindex, 'SCI'] = (nr_gr_single_ret + 0.5 * (nr_gr_first_ret + nr_gr_last_ret))/(nr_all_single_ret + 0.5 * (nr_all_first_ret + nr_all_last_ret))


100000
more than 1
100001
Z:\Thesis\Data\ALS_data\TAVA\481626_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481626_2017_mets.laz   METS
100002
Z:\Thesis\Data\ALS_data\TAVA\481628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481628_2017_mets.laz   TAVA
100003
more than 1
100004
Z:\Thesis\Data\ALS_data\TAVA\481628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481628_2017_mets.laz   TAVA
100005
Z:\Thesis\Data\ALS_data\TAVA\481628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481628_2017_mets.laz   TAVA
100006
Z:\Thesis\Data\ALS_data\TAVA\481628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481628_2017_mets.laz   TAVA
100007
Z:\Thesis\Data\ALS_data\TAVA\481629_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481629_2017_mets.laz   METS
100008
Z:\Thesis\Data\ALS_data\TAVA\481629_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481629_2017_mets.laz   METS
100009
Z:\Thesis\Data\ALS_data\TAVA\481629_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481629_2017_mets.laz   METS
100010
Z:\Thesis\Data\ALS_data\TAV

100098
Z:\Thesis\Data\ALS_data\TAVA\481641_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481641_2017_mets.laz   TAVA
100099
Z:\Thesis\Data\ALS_data\TAVA\481641_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481641_2017_mets.laz   TAVA
100100
Z:\Thesis\Data\ALS_data\TAVA\481641_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481641_2017_mets.laz   TAVA
100101
more than 1
100102
more than 1
100103
more than 1
100104
more than 1
100105
Z:\Thesis\Data\ALS_data\TAVA\481641_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481641_2017_mets.laz   TAVA
100106
more than 1
100107
more than 1
100108
more than 1
100109
Z:\Thesis\Data\ALS_data\TAVA\481645_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481645_2017_mets.laz   TAVA
100110
more than 1
100111
more than 1
100112
Z:\Thesis\Data\ALS_data\TAVA\481646_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481646_2017_mets.laz   TAVA
100113
Z:\Thesis\Data\ALS_data\TAVA\481646_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481646_2017_mets.laz   TAVA
100114
Z:\Thesis\

more than 1
100196
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100197
more than 1
100198
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100199
more than 1
100200
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100201
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100202
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100203
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100204
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100205
Z:\Thesis\Data\ALS_data\TAVA\481657_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481657_2017_mets.laz   METS
100206
more than 1
100

100295
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100296
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100297
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100298
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100299
more than 1
100300
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100301
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100302
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100303
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\481675_2017_mets.laz   METS
100304
Z:\Thesis\Data\ALS_data\TAVA\481675_2019_tava.

100390
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   METS
100391
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   METS
100392
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   METS
100393
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   METS
100394
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   METS
100395
more than 1
100396
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   TAVA
100397
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   TAVA
100398
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\482485_2019_mets.laz   TAVA
100399
Z:\Thesis\Data\ALS_data\TAVA\482485_2021_tava.

...

118517
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118518
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118519
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118520
more than 1
118521
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118522
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118523
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118524
more than 1
118525
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118526
more than 1
118527
Z:\Thesis\Data\ALS_data\TAVA\495594_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495594_2019_mets.laz   TAVA
118528
Z:\Thesi

118608
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118609
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118610
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118611
more than 1
118612
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118613
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118614
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118615
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118616
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495601_2017_mets.laz   TAVA
118617
Z:\Thesis\Data\ALS_data\TAVA\495601_2021_tava.

118699
Z:\Thesis\Data\ALS_data\TAVA\495627_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495627_2017_mets.laz   METS
118700
Z:\Thesis\Data\ALS_data\TAVA\495627_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495627_2017_mets.laz   METS
118701
Z:\Thesis\Data\ALS_data\TAVA\495627_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495627_2017_mets.laz   METS
118702
more than 1
118703
more than 1
118704
more than 1
118705
Z:\Thesis\Data\ALS_data\TAVA\495628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495628_2017_mets.laz   METS
118706
Z:\Thesis\Data\ALS_data\TAVA\495628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495628_2017_mets.laz   METS
118707
Z:\Thesis\Data\ALS_data\TAVA\495628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495628_2017_mets.laz   METS
118708
Z:\Thesis\Data\ALS_data\TAVA\495628_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495628_2017_mets.laz   METS
118709
Z:\Thesis\Data\ALS_data\TAVA\495629_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495629_2017_mets.laz   METS
118710
more tha

118795
Z:\Thesis\Data\ALS_data\TAVA\495642_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495642_2017_mets.laz   TAVA
118796
Z:\Thesis\Data\ALS_data\TAVA\495642_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495642_2017_mets.laz   TAVA
118797
Z:\Thesis\Data\ALS_data\TAVA\495642_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495642_2017_mets.laz   TAVA
118798
Z:\Thesis\Data\ALS_data\TAVA\495642_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495642_2017_mets.laz   TAVA
118799
Z:\Thesis\Data\ALS_data\TAVA\495646_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495646_2017_mets.laz   METS
118800
Z:\Thesis\Data\ALS_data\TAVA\495646_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495646_2017_mets.laz   METS
118801
Z:\Thesis\Data\ALS_data\TAVA\495647_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495647_2017_mets.laz   METS
118802
Z:\Thesis\Data\ALS_data\TAVA\495647_2019_tava.laz 
 Z:\Thesis\Data\ALS_data\METS\495647_2017_mets.laz   METS
118803
Z:\Thesis\Data\ALS_data\TAVA\495647_2019_tava.laz 
 Z:\Thesis\Dat

KeyboardInterrupt: 

In [46]:
pd.set_option('display.max_columns',None)
t = dataframe[beg:end]
t[t['als_sum_all']>0]


,track,seg,timestamp,year,month,beam_nr,beam_t,can_h,n_can_pho,n_topcan_pho,n_ter_pho,can_ter_phoratio,can_pho_rate,ter_pho_rate,openness,dem_h,night,solar_el,snow,brightness,cloud,urban,lon,lat,Hansen_sum,species_majority,species_variety,NR,geometry,ALS_av_h,ALS_98_h,ALS_dns_gap,ALS_cangap,FCI,LCI,SCI,als_sum_all,als_sum_can,als_sum_gr,all_single_ret,can_single_ret,gr_single_ret,all_first,can_first,gr_first,all_last,gr_last,can_last
